In [2]:
# google sheet와 연결
# 참고 자료 : https://pbj0812.tistory.com/184
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = '/opt/ml/project/data/theta-webbing-298612-23be65a42b68.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

In [3]:
#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1Zfhy3xh-2or5dUsSRm9d3yIN4Jhaleh32Rxqejy8hbI/edit#gid=0"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)

#시트 선택하기
sheet_name = "final"
worksheet = doc.worksheet(sheet_name)

In [4]:
import pandas as pd

In [5]:
values = worksheet.get_all_values()
header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)

In [6]:
column_list = ["id","sentence","sentence_with_entity","subject_entity","object_entity","class"]
data = data[column_list]

In [8]:
train_data = data[["id", "sentence", "subject_entity", "object_entity", "class"]]

In [9]:
train_data.rename(columns = {"class" : "label"}, inplace = True)


/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [10]:
train_data.keys()

Index(['id', 'sentence', 'subject_entity', 'object_entity', 'label'], dtype='object')

In [11]:
train_data.head()

,id,sentence,subject_entity,object_entity,label
0,최소기능제품_67,MVP는 사용자가 제품의 특징과 미래를 정의할 수 있도록 사용자의 말을 듣는 공개 ...,"{'start': 0, 'text': 'MVP', 'type': 'poh'}","{'start': 60, 'text': '조기 릴리스', 'type': 'poh'}",no_relation
1,최소기능제품_22,MVP는 핵심적인 기능만 있는 제품이기 때문에 외관상 조악하고 부실하게 보일 수 있...,"{'start': 52, 'text': '창업자', 'type': 'per'}","{'start': 0, 'text': 'MVP', 'type': 'poh'}",product
2,최소기능제품_37,"예를 들어, 2015년에 시드니 대학의 전문가들이 농장과 잡초 관리를 자동화하기 위...","{'start': 14, 'text': '시드니 대학', 'type': 'org'}","{'start': 48, 'text': '리파 로봇', 'type': 'poh'}",product
3,최소기능제품_122,"예약하기 전 본인의 위치 주위에 우버 드라이버들을 보여주고, 승차요금을 예상하고, ...","{'start': 18, 'text': '우버 드라이버', 'type': 'per'}","{'start': 108, 'text': '모바일 앱', 'type': 'poh'}",no_relation
4,최소기능제품_100,Airbnb는 2007년 샌프란시스코에서 시작되었다.,"{'start': 0, 'text': 'Airbnb', 'type': 'org'}","{'start': 14, 'text': '샌프란시스코', 'type': 'loc'}",location


In [15]:
!pwd

/opt/ml/project/data


In [26]:
train_data = pd.read_csv('./data/train/startup.csv')

## Extracting

In [30]:
def preprocessing_dataset(dataset):
  """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
  subject_entity = [sub['text'] for sub in dataset.subject_entity.map(eval)]
  object_entity = [sub['text'] for sub in dataset.object_entity.map(eval)]
  out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],})
  return out_dataset

In [31]:
train_data = preprocessing_dataset(train_data)

In [32]:
train_data.head()

,id,sentence,subject_entity,object_entity,label
0,최소기능제품_67,MVP는 사용자가 제품의 특징과 미래를 정의할 수 있도록 사용자의 말을 듣는 공개 ...,MVP,조기 릴리스,no_relation
1,최소기능제품_22,MVP는 핵심적인 기능만 있는 제품이기 때문에 외관상 조악하고 부실하게 보일 수 있...,창업자,MVP,product
2,최소기능제품_37,"예를 들어, 2015년에 시드니 대학의 전문가들이 농장과 잡초 관리를 자동화하기 위...",시드니 대학,리파 로봇,product
3,최소기능제품_122,"예약하기 전 본인의 위치 주위에 우버 드라이버들을 보여주고, 승차요금을 예상하고, ...",우버 드라이버,모바일 앱,no_relation
4,최소기능제품_100,Airbnb는 2007년 샌프란시스코에서 시작되었다.,Airbnb,샌프란시스코,location
